In [1]:
# Import of relevant python packages
import pandas as pd 
import numpy as np 
import rasterio

In [2]:
# Function to fill matrix with average of closest cells at same distance
# This section is also highlighted in the paper, more specifically
# When the Monfreda map does not have a value in a cell, but the hyde map does
# We search for the closest cells and take the average
def fill_zeros_with_avg(matrix,final_mask):
    filled_matrix = matrix.copy()
    zero_indices = np.argwhere(final_mask)
    for i, j in zero_indices:
        for rr in [5,10,25,50,100,150,200,250]:
            min_row = max(0, i - rr)
            max_row = min(matrix.shape[0], i + rr+1)
            min_col = max(0, j - rr)
            max_col = min(matrix.shape[1], j + rr+1)
            grid = matrix[min_row:max_row, min_col:max_col]
            if np.mean(grid) > 0:
                filled_matrix[i, j] = np.mean(grid)
                break
    output = np.where(final_mask, filled_matrix, matrix)    
    return output 

In [3]:
# Function to calculate the division of monfreda and hyde based on the crop 
def start_eq_1(crop_part,year):
    
    tif_file = "input/{crop_part}_MC.tif".format(crop_part=crop_part)
    tif_file_sum = "input/Sum_MC.tif"
    if crop_part == "Rice": 
        asc_file = "Hyde_3.3_LU/{year}AD_lu/tot_rice{year}AD.asc".format(year=year)
        asc_file_2000 = "Hyde_3.3_LU/2000AD_lu/tot_rice2000AD.asc"
        tif_file_sum = tif_file
    else:
        asc_file = "Hyde_3.3_LU/{year}AD_lu/cropland{year}AD.asc".format(year=year)
        asc_file_2000 = "Hyde_3.3_LU/2000AD_lu/cropland2000AD.asc"
        
    tif_file_max_size = "input/matched_cell_size_ha_map.tiff"
    with rasterio.open(tif_file_max_size) as src:
            max_cell_size = src.read(1)
            metadata = src.profile
        
    with rasterio.open(tif_file) as src:
        MC_Crop_data = src.read(1)
        metadata = src.profile
    
    with rasterio.open(asc_file_2000) as src:
        Hyde_2000_data = src.read(1)
    Hyde_2000_data = np.where(Hyde_2000_data == -9999.0, 0, Hyde_2000_data)

    with rasterio.open(asc_file) as src:
        Hyde_data_year = src.read(1)
    Hyde_data_year = np.where(Hyde_data_year == -9999.0, 0, Hyde_data_year)
    
    with rasterio.open(tif_file_sum) as src:
        monfreda_sum = src.read(1)
        
    Hyde_data_year[Hyde_data_year < 1] = 0
    Hyde_2000_data[Hyde_2000_data < 1] = 0
        
    mask1 = Hyde_data_year > 0
    mask2 = monfreda_sum == 0
    final_mask = np.logical_and(mask1, mask2)
    
    MC_Crop_data = fill_zeros_with_avg(MC_Crop_data,final_mask)
    div_res = np.divide(MC_Crop_data, Hyde_2000_data, 
                        out=np.zeros_like(MC_Crop_data), where=(Hyde_2000_data != 0))
    H_Area_M = div_res * Hyde_data_year
    
    if crop_part == "Rice": 
        H_Area_M = Hyde_data_year
    
    H_Area_M = np.where(H_Area_M > max_cell_size, max_cell_size, H_Area_M)
    
    with rasterio.open("input/EQ1_{crop_part}_{year}.tiff".format(crop_part=crop_part,
                                                                  year=year), 'w', **metadata) as output_raster:
        output_raster.write(H_Area_M,1)
    
    return H_Area_M

In [5]:
# Looping over the various crops and years to obtain that specific crop/year map to move further to step 2 (R script)
crop_data_all = pd.DataFrame({
    "crop_code": ["1_1", "1_2", "1_3", "1_4", "2_1", "2_2", "2_3", "3_1", "3_2", "4", "5", "6", "7"],
    "crop_name": ["Wheat", "Maize", "Rice", "Other Cereals", "Soybean", "Palm Oil fruit", "Other Oilseeds", "Vegetables", "Fruits", "Roots and tubers", "Sugar crops", "Fiber crops", "Other crops"]
})
for year in range(2019, 2020):
    for i,row in crop_data_all.iterrows():
        print(row['crop_name'],year)
        H_Area_M = start_eq_1(row['crop_name'],year)

Wheat 2019
Maize 2019
Rice 2019
Other Cereals 2019
Soybean 2019
Palm Oil fruit 2019
Other Oilseeds 2019
Vegetables 2019
Fruits 2019
Roots and tubers 2019
Sugar crops 2019
Fiber crops 2019
Other crops 2019
